<a href="https://colab.research.google.com/github/kfirbar/sdg-codebase/blob/master/5Ps%20-%20SDG_first_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install transformers
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from transformers import AdamW
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def read_data(filepath):
  data = pd.read_csv(filepath)
  data = data[data["About"].isnull() == False]
  data = data[data["About"] != "0"]
  return data

data = read_data("/content/drive/My Drive/SDG/data/SDG_Companies_ddbb.csv")
data.head()


,Unnamed: 0,Company,Source,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,SDG Goal,About,linkedin,Twitter Handle
0,1.0,https://www.oko.finance/,https://cfamedia.ng/meet-the-eleven-start-ups-...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,OKO (Israel) is crop insurance designed for em...,NaN,NaN
1,2.0,http://www.solarfreeze.co.ke/,https://cfamedia.ng/meet-the-eleven-start-ups-...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Solar Freeze (Kenya) is pioneering mobile cold...,NaN,NaN
2,3.0,Leaf,tech2impact,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"Leaf is a global, virtual bank for vulnerable ...",NaN,NaN
3,4.0,eSolidar,tech2impact,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Platform that brings together those who need h...,NaN,NaN
5,5.0,briteforcesoftware.com,Rainmaking,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"About Us\r\n\r\nBriteforce, Inc was incorporat...",NaN,NaN


In [ ]:
data = data[data["SDG Goal"].isnull() == False]
data.shape

(357, 24)

In [ ]:
def SDGToFivePs(x):
  if x in range(0,7):
    return 'People'
  if x in range(13,16):
    return 'Planet'
  if x in range(7,13):
    return 'Prosperity'
  if x in range(16,18):
    return 'Peace and partnerships'
  return None

In [ ]:
data["five Ps"] = data['SDG Goal'].apply(SDGToFivePs)

In [ ]:
data["Target"] = data["five Ps"]
labels = list(data["Target"].value_counts().keys().values)
print(data["Target"].value_counts())
print(labels)

print(data.shape)

People                    189
Prosperity                129
Peace and partnerships     26
Planet                     13
Name: Target, dtype: int64
['People', 'Prosperity', 'Peace and partnerships', 'Planet']
(357, 26)


In [ ]:
# previous target - SDG goal
'''
data["Target"] = data["SDG Goal"].apply(lambda x: str(x) + "_goal")
# data = data[(data["Target"]=="3.0_goal") | (data["Target"]=="9.0_goal") | 
#            (data["Target"]=="2.0_goal") | (data["Target"]=="1.0_goal") | (data["Target"]=="7.0_goal") |
#            (data["Target"]=="4.0_goal") | (data["Target"]=="6.0_goal")]
labels = list(data["Target"].value_counts().keys().values)
print(data["Target"].value_counts())
print(labels)

print(data.shape)
'''

3.0_goal     44
9.0_goal     36
2.0_goal     35
1.0_goal     30
7.0_goal     29
4.0_goal     28
6.0_goal     27
10.0_goal    26
5.0_goal     25
11.0_goal    19
17.0_goal    13
16.0_goal    13
8.0_goal     11
12.0_goal     8
15.0_goal     7
13.0_goal     5
14.0_goal     1
nan_goal      1
Name: Target, dtype: int64
['3.0_goal', '9.0_goal', '2.0_goal', '1.0_goal', '7.0_goal', '4.0_goal', '6.0_goal', '10.0_goal', '5.0_goal', '11.0_goal', '17.0_goal', '16.0_goal', '8.0_goal', '12.0_goal', '15.0_goal', '13.0_goal', '14.0_goal', 'nan_goal']
(358, 25)


In [ ]:
data.head()

,Unnamed: 0,Company,Source,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,SDG Goal,About,linkedin,Twitter Handle,five Ps,Target
0,1.0,https://www.oko.finance/,https://cfamedia.ng/meet-the-eleven-start-ups-...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,OKO (Israel) is crop insurance designed for em...,NaN,NaN,People,People
1,2.0,http://www.solarfreeze.co.ke/,https://cfamedia.ng/meet-the-eleven-start-ups-...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Solar Freeze (Kenya) is pioneering mobile cold...,NaN,NaN,People,People
2,3.0,Leaf,tech2impact,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"Leaf is a global, virtual bank for vulnerable ...",NaN,NaN,People,People
3,4.0,eSolidar,tech2impact,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Platform that brings together those who need h...,NaN,NaN,People,People
5,5.0,briteforcesoftware.com,Rainmaking,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,"About Us\r\n\r\nBriteforce, Inc was incorporat...",NaN,NaN,People,People


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, data["Target"], test_size=0.15, random_state=42)

In [ ]:
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

303 303
54 54


In [ ]:
class SDGModel(torch.nn.Module):
  def __init__(self, output_size):
    super(SDGModel, self).__init__()
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    self.bert = BertModel.from_pretrained('bert-base-uncased')
#     for index, param in enumerate(self.bert.parameters()):
#         print(index, param.shape)
#         param.requires_grad = False
    self.out1 = torch.nn.Linear(768, 400)
    self.drop = torch.nn.Dropout(0.5)
    self.out2 = torch.nn.Linear(400, output_size)

  def forward(self, text):
    tokenized_text = self.tokenizer.tokenize("[CLS] " + text + " [SEP]")
    indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens]).cuda()
    
    if tokens_tensor.shape[1] > 500:
      tokens_tensor = tokens_tensor[0][0:500].unsqueeze(0)
    outputs = self.bert(tokens_tensor)[0]
    o = torch.nn.ReLU()(self.out1(outputs[0][0]))
    o = self.drop(o)
    return self.out2(o)


In [ ]:
model = SDGModel(len(labels)).cuda()

In [ ]:
model("Hello World")

tensor([-0.1643,  0.0270,  0.0049, -0.0967], device='cuda:0',
       grad_fn=<AddBackward0>)

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
def evaluate_mode(test_data, model):
    model.eval()
    with torch.no_grad():
        y_pred = []
        y_test = test_data["Target"]
        for index, instance in test_data.iterrows():
            out = model(instance["About"].lower())
            y_pred.append(labels[torch.argmax(out).item()])

        model.train()
        return confusion_matrix(y_test, y_pred), accuracy_score(y_test, y_pred)


In [ ]:
n_epochs = 50

for e in range(1, n_epochs+1):
  current_loss = 0
  curr_data = X_train.sample(frac=1)
  print(len(curr_data))
  for index, instance in curr_data.iterrows():
    optimizer.zero_grad()
    out = model(instance["About"].lower())
    loss = criterion(out.view(1, -1), torch.LongTensor([labels.index(instance["Target"])]).cuda())
    loss.backward()
    optimizer.step()
    current_loss += loss.item()
    if index % 30 == 0:
      print(f"Epoch={e}, index={index}, loss={current_loss/30}")
      print(f"{instance['Target']} --> {labels[torch.argmax(out).item()]}")
      current_loss = 0
  print(evaluate_mode(X_test, model))



303
Epoch=1, index=30, loss=0.6876510620117188
People --> Prosperity
Epoch=1, index=1350, loss=0.24976669351259867
Prosperity --> Prosperity
Epoch=1, index=660, loss=0.40594252943992615
People --> People
Epoch=1, index=2190, loss=0.5323708593845368
Peace and partnerships --> People
Epoch=1, index=720, loss=0.7443888386090597
People --> People
Epoch=1, index=210, loss=1.1434335509936016
People --> People
Epoch=1, index=0, loss=2.976578442255656
People --> Prosperity
Epoch=1, index=1710, loss=2.11749388674895
Prosperity --> People
Epoch=1, index=1530, loss=0.4236560632785161
Prosperity --> People
Epoch=1, index=480, loss=0.6208860844373703
People --> People
(array([[ 0,  6,  0,  0],
       [ 0, 24,  0,  5],
       [ 0,  0,  0,  1],
       [ 0, 12,  0,  6]]), 0.5555555555555556)
303
Epoch=2, index=30, loss=2.891339909036954
People --> People
Epoch=2, index=660, loss=0.09963873128096262
People --> People
Epoch=2, index=480, loss=0.5501688083012899
People --> People
Epoch=2, index=2190, los

In [ ]:
evaluate_mode(X_test, model)

(array([[ 5,  0,  0,  1],
        [ 1, 21,  1,  6],
        [ 0,  0,  1,  0],
        [ 0,  6,  1, 11]]), 0.7037037037037037)